In [ ]:
from pathlib import Path
import textwrap
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xarray as xr
from typing import List, Tuple, Dict, Any


strength_cmap = sns.cubehelix_palette(start=0.5, rot=-0.5, as_cmap=True)

from sdm_eurec4a.visulization import (
    set_custom_rcParams,
    set_paper_rcParams,
    adjust_lightness_array,
    label_from_attrs,
    plot_one_one,
)
from sdm_eurec4a import RepositoryPath
from sdm_eurec4a import data_loading

from sdm_eurec4a.reductions import mean_and_stderror_of_mean


default_colors = set_paper_rcParams()
default_dark_colors = adjust_lightness_array(default_colors, 0.75)

RepoPaths = RepositoryPath("levante")

data_dir = RepoPaths.CLEO_data_dir / Path("output_v4.1")
fig_dir = RepoPaths.fig_dir / Path("paper-v4.1")
fig_dir.mkdir(exist_ok=True, parents=False)

In [ ]:
from importlib import reload

reload(data_loading)

<module 'sdm_eurec4a.data_loading' from '/home/m/m301096/repositories/sdm-eurec4a/src/sdm_eurec4a/data_loading.py'>

In [ ]:
microphysics = data_loading.MicrophysicsStyles()

time_slice = slice(1800, 3599)
crop_end_time_slice = slice(0, 3599)

In [ ]:
ds_eulerian = xr.open_dataset(
    data_loading.__eulerian_data_path__(data_dir=data_dir, microphysic="condensation")
)
ds_eulerian = ds_eulerian.sel(time=time_slice)
ds_eulerian = data_loading.__post_process_eulerian_dataset__(ds=ds_eulerian)

ds_conservation = xr.open_dataset(
    data_loading.__conservation_data_path__(data_dir=data_dir, microphysic="condensation")
)
ds_conservation = ds_conservation.sel(time=crop_end_time_slice)
ds_conservation = data_loading.__post_process_conservation_dataset__(
    ds=ds_conservation,
    da_surface_area=ds_eulerian["surface_area"],
    timestep=ds_conservation["time"].diff("time").mean().values,
)

In [ ]:
(ds_eulerian["mass_represented_temporal_mean"] / ds_eulerian["gridbox_volume"]).sel(
    cloud_id=np.random.choice(ds_eulerian["cloud_id"], 1)
).plot()
plt.xscale("log")
plt.xlim(1, None)

(1, np.float64(3023.1801674018375))

In [ ]:
cleo_dataset = data_loading.CleoDataset(
    data_dir=data_dir,
    microphysics=tuple(microphysics),
)
ds, ds_sem = cleo_dataset()
cleo_dataset.normalize_gridboxes()
ds_normalized, ds_normalized_sem = cleo_dataset()

NameError: name 'microphysics' is not defined

# Data and Methods
### Stationary State

In [ ]:
rolling_indices = 5
xlim = np.array((0, 1800))
ylim = np.array((0, 20))

fig, axs = plt.subplots(ncols=2, width_ratios=[1, 0.25], figsize=(7, 3.5))
ax: plt.Axes = axs[0]
ax_hist: plt.Axes = axs[1]
ax_hist.sharey(ax)

x = ds_conservation["time"]
y = -ds_conservation["outflow_precipitation"].transpose("time", ...)
y_rolling = y.rolling(time=rolling_indices, center=True).mean()
x_rolling = x.rolling(time=rolling_indices, center=True).mean()

y_mean, y_sem = mean_and_stderror_of_mean(y.sel(time=time_slice), dims=("time",))

N = y_mean.sizes["cloud_id"]
# Inter-model spread (std of model means)
inter_model_spread = y_mean.std(dim="cloud_id", ddof=1) / np.sqrt(N)
# Individual model uncertainty propagation
individual_model_error = np.sqrt((y_sem**2).sum("cloud_id") / N)
# Total propagated SEM
total_sem = np.sqrt(inter_model_spread**2 + individual_model_error**2)
total_mean = y_mean.mean("cloud_id")

x = x.isel(time=slice(0, -2))
y = y.isel(time=slice(0, -2))

# ax.plot(
#     x,
#     y,
#     color = default_colors[0],
#     alpha = 0.2,
# );
ax.plot(
    x_rolling,
    y_rolling,
    color="grey",
    alpha=0.2,
    label="Condensation",
)
ax.plot(
    x,
    y.mean("cloud_id"),
    color="k",
    alpha=1,
    label="Condensation",
)

hist_bins = np.arange(0, 30, 0.5)
ax_hist.hist(
    y_mean,
    bins=hist_bins,
    orientation="horizontal",
    color="grey",
    alpha=0.2,
)

for _ax in [ax, ax_hist]:
    _ax.set_ylim(0, 30)
    _ax.axhline(total_mean, color="orange", linestyle="--", label="Intercloud mean ± SEM")
    _ax.fill_between(
        _ax.get_xlim(),
        total_mean - total_sem,
        total_mean + total_sem,
        color="orange",
        alpha=0.2,
    )

ax.set_ylim(ylim)
ax.set_ylabel(label_from_attrs(y, name_width=20))
ax.set_xlabel(label_from_attrs(x))
ax.set_xlim(xlim)
ax.set_yticks(np.arange(ylim.min(), ylim.max() + 1, 5))
ax_hist.set_xlabel("Count")

Text(0.5, 0, 'Count')

In [ ]:
rolling_indices = 5
xlim = np.array((0, 1800))
ylim = np.array((0, 100))

fig, axs = plt.subplots(ncols=2, width_ratios=[1, 0.25], figsize=(7, 3.5))
ax: plt.Axes = axs[0]
ax_hist: plt.Axes = axs[1]
ax_hist.sharey(ax)

x = ds_conservation["time"]
y = ds_conservation["inflow_precipitation"].transpose("time", ...)
y_rolling = y.rolling(time=rolling_indices, center=True).mean()
x_rolling = x.rolling(time=rolling_indices, center=True).mean()

y_mean, y_sem = mean_and_stderror_of_mean(y.sel(time=time_slice), dims=("time",))

N = y_mean.sizes["cloud_id"]
# Inter-model spread (std of model means)
inter_model_spread = y_mean.std(dim="cloud_id", ddof=1) / np.sqrt(N)
# Individual model uncertainty propagation
individual_model_error = np.sqrt((y_sem**2).sum("cloud_id") / N)
# Total propagated SEM
total_sem = np.sqrt(inter_model_spread**2 + individual_model_error**2)
total_mean = y_mean.mean("cloud_id")

x = x.isel(time=slice(0, -2))
y = y.isel(time=slice(0, -2))

# ax.plot(
#     x,
#     y,
#     color = default_colors[0],
#     alpha = 0.2,
# );
ax.plot(
    x_rolling,
    y_rolling,
    color="grey",
    alpha=0.2,
    label="Condensation",
)
ax.plot(
    x,
    y.mean("cloud_id"),
    color="k",
    alpha=1,
    label="Condensation",
)

hist_bins = np.arange(0, 30, 0.5)
ax_hist.hist(
    y_mean,
    bins=hist_bins,
    orientation="horizontal",
    color="grey",
    alpha=0.2,
)

for _ax in [ax, ax_hist]:
    _ax.set_ylim(0, 30)
    _ax.axhline(total_mean, color="orange", linestyle="-")
    _ax.fill_between(
        _ax.get_xlim(),
        total_mean - total_sem,
        total_mean + total_sem,
        color="orange",
        alpha=0.2,
    )

ax.set_ylim(ylim)
ax.set_ylabel(label_from_attrs(y, name_width=20))
ax.set_xlabel(label_from_attrs(x))
ax.set_xlim(xlim)
ax.set_yticks(np.arange(ylim.min(), ylim.max() + 1, 5))
# ax.set_yscale('symlog')
ax_hist.set_xlabel("Count")

Text(0.5, 0, 'Count')

# Results

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(5, 5), width_ratios=[1, 0.2], height_ratios=[0.2, 1])

ax_empty = axs[0, 1]
ax_empty.axis("off")
ax_x_hist = axs[0, 0]
ax_y_hist = axs[1, 1]
ax_scatter = axs[1, 0]

ax_x_hist.sharex(ax_scatter)
ax_y_hist.sharey(ax_scatter)

x = -ds["source_energy"]
y = ds["evaporation_fraction"]

x_bins = np.geomspace(1e-3, 1000, 30)
ax_scatter.set_xscale("log")
ax_scatter.set_xlim(x_bins[0], x_bins[-1])

y_bins = np.geomspace(1e-1, 101, 30)
ax_scatter.set_ylim(y_bins[0], y_bins[-1])
ax_scatter.set_yscale("log")

for mp in ["condensation"]:
    ax_scatter.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        **microphysics.get_style(mp),
    )
    ax_x_hist.hist(
        x.sel(microphysics=mp),
        bins=x_bins,
        histtype="step",
        color=microphysics.get_style(mp)["color"],
        lw=2,
    )
    ax_y_hist.hist(
        y.sel(microphysics=mp),
        bins=y_bins,
        histtype="step",
        color=microphysics.get_style(mp)["color"],
        lw=2,
        orientation="horizontal",
    )

ax_scatter.set_xlabel(label_from_attrs(x))
ax_scatter.set_ylabel(label_from_attrs(y))

ax_x_hist.set_ylabel("Counts")
ax_y_hist.set_xlabel("Counts")


for _ax in axs.flatten():
    _ax.grid(linestyle="-", alpha=0.2, color="grey")

fig.tight_layout()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(5, 5), width_ratios=[1, 0.5], height_ratios=[0.5, 1])

ax_empty = axs[0, 1]
ax_empty.axis("off")
ax_x_hist = axs[0, 0]
ax_y_hist = axs[1, 1]
ax_scatter = axs[1, 0]

ax_x_hist.sharex(ax_scatter)
ax_y_hist.sharey(ax_scatter)

x = -ds["source_energy"]
y = ds["evaporation_fraction"]

x_bins = np.linspace(0, 1000, 30)
# ax_scatter.set_xscale("log")

y_bins = np.linspace(0, 101, 30)
# ax_scatter.set_yscale("log")

for mp in ["condensation"]:
    ax_scatter.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        **microphysics.get_style(mp),
    )
    ax_x_hist.hist(
        x.sel(microphysics=mp),
        bins=x_bins,
        histtype="step",
        color=microphysics.get_style(mp)["color"],
        lw=2,
    )
    ax_y_hist.hist(
        y.sel(microphysics=mp),
        bins=y_bins,
        histtype="step",
        color=microphysics.get_style(mp)["color"],
        lw=2,
        orientation="horizontal",
    )

ax_scatter.set_xlabel(label_from_attrs(x, name_width=20))
ax_scatter.set_ylabel(label_from_attrs(y))

ax_x_hist.set_ylabel("Counts")
ax_y_hist.set_xlabel("Counts")

x_limits = ax_x_hist.get_ylim()
y_limits = ax_y_hist.get_xlim()
limits = [0, max(x_limits[1], y_limits[1])]
ax_x_hist.set_ylim(limits)
ax_y_hist.set_xlim(limits)


# dx = x_bins[-1] - x_bins[0]
# dy = y_bins[-1] - y_bins[0]
# adjuster = 0.02

# ax_scatter.set_xlim(x_bins[0] - adjuster * dx, x_bins[-1] + adjuster * dx)
# ax_scatter.set_ylim(y_bins[0] - adjuster * dy, y_bins[-1] + adjuster * dy)
ax_scatter.set_xlim(x_bins[0], x_bins[-1])
ax_scatter.set_ylim(y_bins[0], y_bins[-1])


for _ax in axs.flatten():
    _ax.grid(linestyle="-", alpha=0.2, color="grey")

fig.tight_layout()